In [1]:
import matplotlib.pyplot as plt
import numpy as np
from os import listdir, sys
from os.path import isfile, join
import tensorflow as tf
from scipy import stats
import time
from imp import reload

%matplotlib inline

In [2]:
read_file_path = "/home/ch194765/repos/atlas-active-learning/"

sys.path.insert(0, read_file_path)
import prep_dat

# Playing with TF Gradients

In [2]:
# input
X = tf.placeholder("float", [3, None])
y = tf.placeholder("float", [2, None])

# layer one
W_1 = tf.Variable(tf.ones([3,3]))
b_1 = tf.Variable(tf.zeros([3,1]))
z_1 = tf.add(tf.matmul(W_1, X), b_1)
a_1 = tf.nn.sigmoid(z_1)

# layer two (last layer)
W_2 = tf.Variable(tf.ones([2,3]))
b_2 = tf.Variable(tf.zeros([2,1]))
z_2 = tf.add(tf.matmul(W_2, a_1), b_2)
a_2 = tf.sigmoid(z_2)   # (output)

# loss function
J = tf.multiply(tf.reduce_sum(tf.pow(tf.subtract(y, a_2), 2)), .5)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


In [3]:
Xin = np.array([[1., 0.], [0., 1.], [0., 0.]])
yin = np.array([[1., 0.], [0., 1.]])
grad_vals = tf.gradients(J, [W_1, b_1, W_2, b_2])
sess.run(grad_vals, feed_dict={X:Xin, y:yin})

[array([[ 0.014189,  0.014189,  0.      ],
        [ 0.014189,  0.014189,  0.      ],
        [ 0.014189,  0.014189,  0.      ]], dtype=float32),
 array([[ 0.02837801],
        [ 0.02837801],
        [ 0.02837801]], dtype=float32),
 array([[ 0.05275872,  0.05275872,  0.05275872],
        [ 0.05275872,  0.05275872,  0.05275872]], dtype=float32),
 array([[ 0.07216756],
        [ 0.07216756]], dtype=float32)]

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(56)
G = optimizer.compute_gradients(J, [W_1, b_1, W_2, b_2])
Gs = sess.run(G, feed_dict={X:Xin, y:yin})
#Gs

# Try Some Active Learning on an MNIST Example

In [3]:
# load the data from TF 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Here, we don't use all the training data of the `mnist` object since we need to leave some part of the training unlabeled to do querying. Hence, we manually take a portion of the training as the initial labeled data set, and use the rest for evaluating our active learning method.

In [4]:
# number of the initial labeled data set
init_size = 200
batch_size = 50
epochs = 10

# randomly selecting that many samples from the training data set
train_size = mnist.train.images.shape[0]
rand_inds = np.random.permutation(train_size)
init_inds = rand_inds[:init_size]
unlabeled_inds = rand_inds[init_size:]
#
init_train_images = mnist.train.images[init_inds, :]
init_train_labels = mnist.train.labels[init_inds, :]
pool_images = mnist.train.images[unlabeled_inds, :]
pool_labels = mnist.train.labels[unlabeled_inds, :]

# manually creating batches for initial training
batch_inds = prep_dat.gen_batch_inds(init_size, batch_size)
batch_of_data = prep_dat.gen_batch_matrices(init_train_images, batch_inds)
batch_of_labels = prep_dat.gen_batch_matrices(init_train_labels, batch_inds)

Now, use the initial training data set to train a simple NN over the data using cross-entropy or any other loss function. 

In [5]:
sess = tf.InteractiveSession()

# input and output placeholders
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# parameters
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

# output should be a normalized 
y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

# optimization iteration
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# doing the optimization
for _ in range(epochs):
    for i in range(len(batch_inds)):
      train_step.run(feed_dict={x: batch_of_data[i], 
                                y_: batch_of_labels[i]})


In [6]:
# Evaluation:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Initial Accuracy: %.4f" % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


Initial Accuracy: 0.7894


In [7]:
sess.run(y, feed_dict={x: mnist.train.images, y_: mnist.train.labels})


array([[-0.09420905, -2.83136511,  2.77282429, ...,  2.36155581,
        -1.51649237,  2.39522862],
       [-1.19596696, -0.02359757,  0.24169362, ..., -0.06306656,
         3.17758489,  0.50698704],
       [-1.96824872, -3.44915843, -1.95908368, ...,  1.35921288,
         0.86376554,  1.69012916],
       ..., 
       [-0.41222823, -1.15216482, -2.76455379, ...,  0.90987086,
         1.27789843,  0.65098399],
       [ 1.25641358, -2.33889055,  0.9237808 , ..., -1.80518234,
        -1.78872836,  0.1966071 ],
       [ 0.35340229, -1.45301032, -0.23499081, ..., -0.50632918,
         3.0099678 ,  1.52604902]], dtype=float32)

## Active Learning

In [ ]:
# log-posteriors (or log-likelihood wrt the parameters)
log_posteriors = tf.log(tf.nn.softmax(y))

# T